In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle

from lasso import Parameters, ModelRoadwayNetwork
from network_wrangler import RoadwayNetwork


In [2]:
input_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'client_met_council_network')
pickle_dir = os.path.join(input_dir, 'network_standard')

lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'lasso')

In [3]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-09-11 20:47:59, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso


In [4]:
version_00_pickle_file_name = os.path.join(pickle_dir, 'working_scenario_00.pickle')
v_01_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

In [5]:
net = v_01_scenario.road_net

In [6]:
mrcc_shst_ref_df = pd.read_csv(parameters.mrcc_shst_data)
mrcc_gdf = gpd.read_file(parameters.mrcc_roadway_class_shape)
mrcc_roadway_class_variable_shp = 'ROUTE_SYS'

In [7]:
widot_shst_ref_df = ModelRoadwayNetwork.read_match_result(parameters.widot_shst_data)
widot_gdf = gpd.read_file(parameters.widot_roadway_class_shape)
widot_gdf["LINK_ID"] = range(1, 1 + len(widot_gdf))
widot_roadway_class_variable_shp = "RDWY_CTGY_"

In [8]:
join_gdf = ModelRoadwayNetwork.get_attribute(
            net.links_df,
            "shstGeometryId",
            mrcc_shst_ref_df,
            mrcc_gdf,
            mrcc_roadway_class_variable_shp,
        )

In [9]:
if "mrcc_id" in net.links_df.columns:
    join_gdf.drop(["source_link_id"], axis=1, inplace=True)
else:
    join_gdf.rename(columns={"source_link_id" : "mrcc_id"}, inplace=True)

join_gdf.columns

Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'lanes', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry', 'ROUTE_SYS', 'mrcc_id'],
      dtype='object')

In [10]:
join_gdf = ModelRoadwayNetwork.get_attribute(
            join_gdf,
            "shstGeometryId",
            widot_shst_ref_df,
            widot_gdf,
            widot_roadway_class_variable_shp,
        )

In [11]:
mrcc_asgngrp_crosswalk_df = pd.read_csv(parameters.mrcc_assgngrp_dict, dtype={mrcc_roadway_class_variable_shp: str})
osm_asgngrp_crosswalk_df = pd.read_csv(parameters.osm_assgngrp_dict)
widot_asgngrp_crosswak_df = pd.read_csv(parameters.widot_assgngrp_dict)

In [12]:
join_gdf = pd.merge(
            join_gdf,
            mrcc_asgngrp_crosswalk_df.rename(
                columns={
                "assign_group": "assignment_group_mrcc",
                "roadway_class": "roadway_class_mrcc"
                }
            ),
            how="left",
            on=mrcc_roadway_class_variable_shp,
        )

In [13]:
join_gdf.assignment_group_mrcc.value_counts()

50.0    157573
7.0      36013
6.0      32797
4.0       4156
2.0       1404
1.0        821
11.0       121
8.0          2
Name: assignment_group_mrcc, dtype: int64

In [14]:
join_gdf.roadway_class_mrcc.value_counts()

50.0    159019
40.0     40385
30.0     22259
20.0      6130
60.0      4156
10.0       823
70.0       115
Name: roadway_class_mrcc, dtype: int64

In [15]:
join_gdf.head()

,model_link_id,osm_link_id,shstReferenceId,shstGeometryId,shape_id,u,v,A,B,locationReferences,...,trn_priority,ttime_assert,geometry,ROUTE_SYS,mrcc_id,RDWY_CTGY_,source_link_id,ROUTE_SYS_def,assignment_group_mrcc,roadway_class_mrcc
0,1,18235015,bd54d966087015133c38ff67a29eadce,982678f207eaf32a660f7c73e8e9781c,3147b4e2f55c107918d074e659ba97ca,460324884,188052194,3101,129359,"[{'sequence': 1, 'point': [-93.5036781, 44.780...",...,0,0.0,"LINESTRING (-93.50368 44.78098, -93.50367 44.7...",10,146378.0,NaN,NaN,Municipal Street,50.0,50.0
1,3,640843276,049eec6bc0999f122acda4d556379931,c24f3c636cdb206bc5af751a49e0ea3a,7283c7ef7b49187f8a867cd397f53a59,460324884,188042431,3101,124942,"[{'sequence': 1, 'point': [-93.5036781, 44.780...",...,0,0.0,"LINESTRING (-93.50368 44.78098, -93.50100 44.7...",05,146377.0,NaN,NaN,Municipal-State Aid Street,7.0,40.0
2,4,640843276,09fd881762b82971980039a538165465,1a37e2d4c352b639434827a2f55f478d,d6903f9ff3a4782ec3f172783b812171,460324887,460324884,3102,3101,"[{'sequence': 1, 'point': [-93.505235, 44.7810...",...,0,0.0,"LINESTRING (-93.50523 44.78107, -93.50368 44.7...",05,146377.0,NaN,NaN,Municipal-State Aid Street,7.0,40.0
3,6,"24474311,26128727",29994043e566e525162e1ccb53b3ee64,26eb0c281aa2deea46eabb542927827f,060efc5ede6d2bc1a15213c3b76228e8,2010120217,266072985,3103,82316,"[{'sequence': 1, 'point': [-93.8066951, 45.037...",...,0,0.0,"LINESTRING (-93.80670 45.03786, -93.80505 45.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,26128737,6e9311064a72d89241500dee95573f93,14d8797dc30831685b213bfa6a2657ee,a8521dc30edead763ec6db80725b2040,2010120217,286055147,3103,113472,"[{'sequence': 1, 'point': [-93.8066951, 45.037...",...,0,0.0,"LINESTRING (-93.80670 45.03786, -93.80330 45.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
join_gdf = pd.merge(
            join_gdf,
            osm_asgngrp_crosswalk_df.rename(
                columns={
                "assign_group": "assignment_group_osm",
                "roadway_class": "roadway_class_osm"
                }
            ),
            how="left",
            on="roadway",
        )

In [17]:
join_gdf.assignment_group_osm.value_counts()

50.0    215097
7.0      78965
6.0      28088
1.0       5294
15.0      2512
11.0         2
Name: assignment_group_osm, dtype: int64

In [18]:
join_gdf.roadway_class_osm.value_counts()

50.0    215097
40.0     78967
30.0     19927
20.0     10673
10.0      5294
Name: roadway_class_osm, dtype: int64

In [19]:
join_gdf.head()

,model_link_id,osm_link_id,shstReferenceId,shstGeometryId,shape_id,u,v,A,B,locationReferences,...,geometry,ROUTE_SYS,mrcc_id,RDWY_CTGY_,source_link_id,ROUTE_SYS_def,assignment_group_mrcc,roadway_class_mrcc,assignment_group_osm,roadway_class_osm
0,1,18235015,bd54d966087015133c38ff67a29eadce,982678f207eaf32a660f7c73e8e9781c,3147b4e2f55c107918d074e659ba97ca,460324884,188052194,3101,129359,"[{'sequence': 1, 'point': [-93.5036781, 44.780...",...,"LINESTRING (-93.50368 44.78098, -93.50367 44.7...",10,146378.0,NaN,NaN,Municipal Street,50.0,50.0,50.0,50.0
1,3,640843276,049eec6bc0999f122acda4d556379931,c24f3c636cdb206bc5af751a49e0ea3a,7283c7ef7b49187f8a867cd397f53a59,460324884,188042431,3101,124942,"[{'sequence': 1, 'point': [-93.5036781, 44.780...",...,"LINESTRING (-93.50368 44.78098, -93.50100 44.7...",05,146377.0,NaN,NaN,Municipal-State Aid Street,7.0,40.0,7.0,40.0
2,4,640843276,09fd881762b82971980039a538165465,1a37e2d4c352b639434827a2f55f478d,d6903f9ff3a4782ec3f172783b812171,460324887,460324884,3102,3101,"[{'sequence': 1, 'point': [-93.505235, 44.7810...",...,"LINESTRING (-93.50523 44.78107, -93.50368 44.7...",05,146377.0,NaN,NaN,Municipal-State Aid Street,7.0,40.0,7.0,40.0
3,6,"24474311,26128727",29994043e566e525162e1ccb53b3ee64,26eb0c281aa2deea46eabb542927827f,060efc5ede6d2bc1a15213c3b76228e8,2010120217,266072985,3103,82316,"[{'sequence': 1, 'point': [-93.8066951, 45.037...",...,"LINESTRING (-93.80670 45.03786, -93.80505 45.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,50.0
4,7,26128737,6e9311064a72d89241500dee95573f93,14d8797dc30831685b213bfa6a2657ee,a8521dc30edead763ec6db80725b2040,2010120217,286055147,3103,113472,"[{'sequence': 1, 'point': [-93.8066951, 45.037...",...,"LINESTRING (-93.80670 45.03786, -93.80330 45.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,50.0


In [20]:
join_gdf = pd.merge(
            join_gdf,
            widot_asgngrp_crosswak_df.rename(
                columns={
                "assign_group": "assignment_group_widot",
                "roadway_class": "roadway_class_widot",
                }
            ),
            how="left",
            on=widot_roadway_class_variable_shp,
        )

In [21]:
join_gdf.assignment_group_widot.value_counts()

50.0    18430
6.0      6061
4.0       230
7.0        98
1.0        27
Name: assignment_group_widot, dtype: int64

In [22]:
join_gdf.roadway_class_widot.value_counts()

50.0    18430
20.0     6061
60.0      230
40.0       98
10.0       27
Name: roadway_class_widot, dtype: int64

In [23]:
join_gdf.loc[join_gdf['RDWY_CTGY_'].notna()].head()

,model_link_id,osm_link_id,shstReferenceId,shstGeometryId,shape_id,u,v,A,B,locationReferences,...,RDWY_CTGY_,source_link_id,ROUTE_SYS_def,assignment_group_mrcc,roadway_class_mrcc,assignment_group_osm,roadway_class_osm,RDWY_def,assignment_group_widot,roadway_class_widot
128,175,353980874,ccfae70b608b015e143472c4c13cee8b,99ac42d7e842162d355f6e3ed5762d5f,229b8948c7af160c17da7f566ea413b9,3597666809,232019832,3206,96508,"[{'sequence': 1, 'point': [-92.223427, 45.3956...",...,4.0,23020.0,NaN,NaN,NaN,7.0,40.0,County Trunk Highway,6.0,20.0
206,289,21584329,4f786ed9746fd9b00a876a9b711dac5f,d4b379d8d1e609ac523d5ed0d45f163a,93dac4fcb48504b9fee537228812e9c1,2647657471,287787787,3264,161072,"[{'sequence': 1, 'point': [-92.6315191, 44.887...",...,5.0,1868.0,NaN,NaN,NaN,50.0,50.0,Municipal Road,50.0,50.0
207,290,259363992,bce3449c4842f8809e0922b2b6481c5a,fb1db31b0be86fe2dca0d91efe9c3b0c,bb1425f1d064e1f56dd65476fe9c64b7,2647657471,3192229488,3264,132062,"[{'sequence': 1, 'point': [-92.6315191, 44.887...",...,5.0,24760.0,NaN,NaN,NaN,50.0,50.0,Municipal Road,50.0,50.0
228,322,21448740,2866c5e9ca3b70dfbcaeb8346f32a819,7508be3cd9912a3d15ad1ac2c47a7e1b,03c752f9376f9571aa99b333af2668fe,230690296,232029954,3283,102538,"[{'sequence': 1, 'point': [-92.506786, 45.7285...",...,5.0,13177.0,NaN,NaN,NaN,50.0,50.0,Municipal Road,50.0,50.0
234,330,21448750,5a65ec50ff6ce3aa3ef850db39b0a423,604912ad0d91fb503861c4f4a42254f7,7a890ba061d6592ad14fd9a653a9da23,230690410,230690413,3286,3287,"[{'sequence': 1, 'point': [-92.247339, 45.7259...",...,5.0,12561.0,NaN,NaN,NaN,50.0,50.0,Municipal Road,50.0,50.0


In [34]:
join_gdf['centroidconnect'] = 0

In [38]:
road_class_variable_name="roadway_class"
def _set_roadway_class(x):
            try:
                if x.centroidconnect == 1:
                    return 99
                elif x.bus_only == 1:
                    return 50
                elif x.rail_only == 1:
                    return 100
                elif x.drive_access == 0:
                    return 101
                elif x.roadway_class_mrcc > 0:
                    return int(x.roadway_class_mrcc)
                elif x.roadway_class_widot > 0:
                    return int(x.roadway_class_widot)
                else:
                    return int(x.roadway_class_osm)
            except:
                return 0

join_gdf[road_class_variable_name] = join_gdf.apply(lambda x: _set_roadway_class(x), axis=1)

In [39]:
join_gdf.head()

,model_link_id,osm_link_id,shstReferenceId,shstGeometryId,shape_id,u,v,A,B,locationReferences,...,ROUTE_SYS_def,assignment_group_mrcc,roadway_class_mrcc,assignment_group_osm,roadway_class_osm,RDWY_def,assignment_group_widot,roadway_class_widot,centroidconnect,roadway_class
0,1,18235015,bd54d966087015133c38ff67a29eadce,982678f207eaf32a660f7c73e8e9781c,3147b4e2f55c107918d074e659ba97ca,460324884,188052194,3101,129359,"[{'sequence': 1, 'point': [-93.5036781, 44.780...",...,Municipal Street,50.0,50.0,50.0,50.0,NaN,NaN,NaN,0,50
1,3,640843276,049eec6bc0999f122acda4d556379931,c24f3c636cdb206bc5af751a49e0ea3a,7283c7ef7b49187f8a867cd397f53a59,460324884,188042431,3101,124942,"[{'sequence': 1, 'point': [-93.5036781, 44.780...",...,Municipal-State Aid Street,7.0,40.0,7.0,40.0,NaN,NaN,NaN,0,40
2,4,640843276,09fd881762b82971980039a538165465,1a37e2d4c352b639434827a2f55f478d,d6903f9ff3a4782ec3f172783b812171,460324887,460324884,3102,3101,"[{'sequence': 1, 'point': [-93.505235, 44.7810...",...,Municipal-State Aid Street,7.0,40.0,7.0,40.0,NaN,NaN,NaN,0,40
3,6,"24474311,26128727",29994043e566e525162e1ccb53b3ee64,26eb0c281aa2deea46eabb542927827f,060efc5ede6d2bc1a15213c3b76228e8,2010120217,266072985,3103,82316,"[{'sequence': 1, 'point': [-93.8066951, 45.037...",...,NaN,NaN,NaN,50.0,50.0,NaN,NaN,NaN,0,50
4,7,26128737,6e9311064a72d89241500dee95573f93,14d8797dc30831685b213bfa6a2657ee,a8521dc30edead763ec6db80725b2040,2010120217,286055147,3103,113472,"[{'sequence': 1, 'point': [-93.8066951, 45.037...",...,NaN,NaN,NaN,50.0,50.0,NaN,NaN,NaN,0,50
